In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [27]:
!pip freeze | grep pandas

pandas==2.2.3


In [2]:
!python -V

Python 3.13.1


In [ ]:
import pickle
import numpy as np
import pandas as pd

In [22]:
year = 2023
month = 3
taxi_type = 'yellow'

output_file = f'./output/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

In [4]:
with open('./models/lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = pd.to_datetime(df["tpep_dropoff_datetime"]) - pd.to_datetime(df["tpep_pickup_datetime"])
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
np.std(y_pred)

np.float64(13.42015339860258)

In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [24]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
# df_result['lpep_pickup_datetime'] = df['tpep_pickup_datetime']
# df_result['PULocationID'] = df['PULocationID']
# df_result['DOLocationID'] = df['DOLocationID']
# df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
# df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
# df_result['model_version'] = run_id

In [25]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [26]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)